In [1]:
import pandas as pd
import datetime
import matplotlib
import numpy as np
import warnings
import itertools
import statsmodels.api as sm
from pandas import Series
from pandas import datetime
from matplotlib import pyplot as plt
from statsmodels.tsa.arima_model import ARIMA
from pandas.plotting import autocorrelation_plot
from statsmodels.graphics.tsaplots import plot_pacf
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
plt.style.use('fivethirtyeight')
%matplotlib inline
 
production=pd.read_pickle("./processed/production_all_dates_and_variables.pkl")
keep_these=['Wind_KWH',
            'Solar_KWH',
            'Wind_Speed_AT_WINDFARM',
            'Solar_Elevation',
            'Cloud_Cover_Fraction',
            'Dew_Point',
            'Humidity_Fraction',
            'Precipitation',
            'Pressure',
            'Temperature',
            'Visibility',
            'Wind_Speed_AT_SOLARRAY']
production=production[keep_these]
#fill the date and time gaps
production = production.resample("60min").asfreq()
production.to_pickle("./processed/production_full_date_range_pre_impute.pkl")



In [2]:
"""
Implement an imputation scheme for each variable.


Dependent Variables - no imputation
solar - if nighttime, zero
wind - ???
=========================================
'Wind_KWH',
'Solar_KWH',

Dependent Variables - impute as described
CHECK SPARSITY OF VALUES PRIOR TO IMPUTING
=========================================
'Wind_Speed_AT_WINDFARM', - high variance, 
'Solar_Elevation', # Removing, as there is no good way to aggregate this by day, nor is it necessarily relevant except
                     as a performance measure of the solar array, which is not under analysis.
'Cloud_Cover_Fraction', - Average
'Dew_Point', - Average
'Humidity_Fraction', - Average
'Precipitation', - Average
'Pressure', - Average
'Temperature', - Average
'Visibility', - Average
'Wind_Speed_AT_SOLARRAY' - Average

Note: We should do inter-day analysis as well, if possible.

""" 

# Create dataset for seasonality of solar KWH by day.

solar_prod=production['Solar_KWH']
solar_prod_by_day = solar_prod.resample("D").sum().to_frame('Solar_KWH')
solar_prod_by_day.to_pickle("./processed/solar_prod_by_day.pkl")

independent_vars=['Cloud_Cover_Fraction',
                  'Dew_Point',
                  'Humidity_Fraction',
                  'Precipitation',
                  'Pressure',
                  'Temperature',
                  'Visibility',
                  'Wind_Speed_AT_SOLARRAY']

indep_vars=production[independent_vars]
for field in independent_vars:
    # consumption_master = consumption_master.join(calendar, how='outer', sort=True)
    # For each field, aggregate by day, then join to solar_prod
    tmp=pd.DataFrame(data=indep_vars[field])
    tmp.dropna(axis=0, inplace=True)
    tmp=tmp.loc[tmp[field]!='nan'] # How we got text 'nan' in here, I do not know, but remove them.
    tmp=pd.to_numeric(tmp[field], downcast='float')
    # Averaging the remaining values by day should yield a useful metric for modeling, even if
    # it would be meaningless from a real weather-reporting context. 
    # We are imposing the same limitation on all variables, so should still be representative.
    tmp=tmp.resample("D").mean().to_frame(field)
    #print(tmp.head())
    solar_prod_by_day=solar_prod_by_day.join(tmp, how='left', sort=True)
    
#print(solar_prod_by_day.head())


# Now, we need to interpolate missing values.

def interpolate_gaps(values, limit=None):
    """
    Fill gaps using linear interpolation, optionally only fill gaps up to a
    size of `limit`, courtesy of StackOVerflow:
    https://stackoverflow.com/questions/36455083/working-with-nan-values-in-matplotlib
    """
    values = np.asarray(values)
    i = np.arange(values.size)
    valid = np.isfinite(values)
    filled = np.interp(i, i[valid], values[valid])

    if limit is not None:
        invalid = ~valid
        for n in range(1, limit+1):
            invalid[:-n] &= invalid[n:]
        filled[invalid] = np.nan
    return filled


columns={}
dates=solar_prod_by_day.index.values
columns['Date']=dates
for field in list(solar_prod_by_day):
    raw=solar_prod_by_day[field].values
    filled = interpolate_gaps(raw, limit=2)
    columns[field]=filled

solar_prod_by_day_fnl=pd.DataFrame.from_dict(columns)
solar_prod_by_day_fnl.set_index('Date', inplace=True)
solar_prod_by_day_fnl.to_pickle("./processed/solar_prod_by_day_w_interpolation.pkl")
print(solar_prod_by_day_fnl.head())


                Solar_KWH  Cloud_Cover_Fraction  Dew_Point  Humidity_Fraction  \
Date                                                                            
2010-01-04   20084.160000              1.000000  -8.300000           0.698400   
2010-01-05  111402.100000              1.000000  -7.000000           0.789400   
2010-01-06  163901.380000              1.000000  -6.950000           0.768500   
2010-01-07   95813.460000              0.985714  -6.442857           0.865729   
2010-01-08  116167.524153              0.987500 -11.062500           0.723737   

            Precipitation    Pressure  Temperature  Visibility  \
Date                                                             
2010-01-04       0.000000  992.050049    -3.600000   16.093000   
2010-01-05       0.000000  992.112488    -3.875000   15.087251   
2010-01-06       0.000000  993.799988    -3.475000   16.093000   
2010-01-07       0.071429  991.649963    -4.528572   11.954858   
2010-01-08       0.000000  996.01251

In [ ]:
# From: 
# https://www.digitalocean.com/community/tutorials/a-guide-to-time-series-forecasting-with-arima-in-python-3
# Significantly adapted, but the iterative approach and some code is a direct-use.

def find_best_model(train_years, test_years, df, exogenous_variables, endogenous_variable):
    train_data=df[endogenous_variable].loc[(df.index.year.isin(train_years))]
    exog=df[exogenous_variables].loc[(df.index.year.isin(train_years))].values
    train=train_data.values

    test_data=df[endogenous_variable].loc[(df.index.year.isin(test_years))]
    test=test_data.values

    # Define the p, d and q parameters to take any value between 0 and 2
    p = d = q = range(0, 3)

    # Generate all different combinations of p, q and q triplets
    pdq = list(itertools.product(p, d, q))

    # Generate all different combinations of seasonal p, q and q triplets
    # Since we are daily data, m=360, suggesting a yearly cycle.
    seasonal_pdq = [(x[0], x[1], x[2], 360) for x in list(itertools.product(p, d, q))]
    
    warnings.filterwarnings("ignore") # specify to ignore warning messages
    count=1
    for param in pdq:
        for param_seasonal in seasonal_pdq:
            try:
                mod = sm.tsa.statespace.SARIMAX(train,
                                                #exog=exog,
                                                order=param,
                                                seasonal_order=param_seasonal,
                                                enforce_stationarity=False,
                                                enforce_invertibility=False)

                results = mod.fit()
                if count==1:
                    tmp_best=results.aic
                    count+=1
                else:
                    if tmp_best > results.aic:
                        tmp_best=results.aic
                        tmp_best_param=param
                        tmp_best_seasonal=param_seasonal
                        count+=1
                    else:
                        count+=1
                    
                print('ARIMA{}x{} - AIC:{} - best AIC: {} - Iteration: {}'.format(param,
                                                                                    param_seasonal,
                                                                                    results.aic,
                                                                                    tmp_best,
                                                                                    count))
            
            except Exception as e:
                print('Exception: {} ARIMA{}x{} - Iteration: {}'.format(e,
                                                                          param,
                                                                          param_seasonal,
                                                                          count))
                continue
                
    return(tmp_best, tmp_best_param, tmp_best_seasonal)


# Lets split the data into 3 years train, one test, across years. Assumption: That any long-
# term weather trend will not manifest materially within a three year timeframe.

years=[2010, 2011, 2012, 2013]

# Keep: Visibility, Temperature, and Pressure - Pressure has NaN, dropping it.
exogenous_variables=['Visibility', 'Temperature']

#for scenario in scenarios:
for iteration in range(0, len(years)):
    list_len=len(years)
    train_years=years[1:len(years)]
    test_year=[]
    test_year.append(years[0])
    print("Iteration: {} Test: {} Train: {}".format(iteration, test_year, train_years))
    tmp_best, tmp_best_param, tmp_best_seasonal = find_best_model(train_years,
                                                                  test_year,
                                                                  solar_prod_by_day_fnl,
                                                                  exogenous_variables,
                                                                  'Solar_KWH')

    print("Best param: {} Best Seasonal Param: {} best AIC: {}".format(tmp_best_param,
                                                                       tmp_best_seasonal,
                                                                       tmp_best))
    years.append(years.pop(0))

    


Iteration: 0 Test: [2010] Train: [2011, 2012, 2013]
ARIMA(0, 0, 0)x(0, 0, 0, 360) - AIC:33039.11303362222 - best AIC: 33039.11303362222 - Iteration: 2
ARIMA(0, 0, 0)x(0, 0, 1, 360) - AIC:263711263.75175545 - best AIC: 33039.11303362222 - Iteration: 3
Exception: maxlag should be < nobs ARIMA(0, 0, 0)x(0, 0, 2, 360) - Iteration: 3
ARIMA(0, 0, 0)x(0, 1, 0, 360) - AIC:21514.065046649273 - best AIC: 21514.065046649273 - Iteration: 4
Exception: maxlag should be < nobs ARIMA(0, 0, 0)x(0, 1, 1, 360) - Iteration: 4
Exception: maxlag should be < nobs ARIMA(0, 0, 0)x(0, 1, 2, 360) - Iteration: 4
Exception:  ARIMA(0, 0, 0)x(0, 2, 0, 360) - Iteration: 4
Exception: maxlag should be < nobs ARIMA(0, 0, 0)x(0, 2, 1, 360) - Iteration: 4
Exception: maxlag should be < nobs ARIMA(0, 0, 0)x(0, 2, 2, 360) - Iteration: 4
ARIMA(0, 0, 0)x(1, 0, 0, 360) - AIC:21403.028964338708 - best AIC: 21403.028964338708 - Iteration: 5
ARIMA(0, 0, 0)x(1, 0, 1, 360) - AIC:263708104.22930986 - best AIC: 21403.028964338708 - It

In [ ]:
mod = sm.tsa.statespace.SARIMAX(train,
                                order=tmp_best_param,
                                seasonal_order=tmp_best_seasonal,
                                enforce_stationarity=False,
                                enforce_invertibility=False)

results = mod.fit()

print(results.summary().tables[1])

In [ ]:
results.plot_diagnostics(figsize=(15, 12))
plt.show()
#imgname="SARIMA_train_{}_test_{}_param_{}_seasonal_{}".format(2010, 2013, tmp_best_param, tmp_best_seasonal)
#results.plot_diagnostics(figsize=(15, 12)).savefig("./Images/" + imgname + '.png')

In [ ]:
#print(solar_prod_by_day_fnl.corr())
import seaborn as sns

f, ax = plt.subplots(figsize=(10, 8))
corr = solar_prod_by_day_fnl.corr()
sns.heatmap(corr,
            mask=np.zeros_like(corr,
                               dtype=np.bool),
            cmap=sns.diverging_palette(220,
                                       10,
                                       as_cmap=True),
            square=True, ax=ax);

In [ ]:
# Keep Visibility, Temperature, and Pressure

In [ ]:
"""
Our primary concern is to ensure that the residuals of our model are uncorrelated and 
normally distributed with zero-mean. If the seasonal ARIMA model does not satisfy these properties, 
it is a good indication that it can be further improved.

In this case, our model diagnostics suggests that the model residuals are roughly normally 
distributed based on the following:

In the top right plot, we see that the red KDE line does follows the N(0,1) line, which 
is the standard notation for a normal distribution with mean 0 and standard deviation of 1), except for a 
spike between 0 and 1. This is a good indication that the residuals are roughly normally distributed.

The qq-plot on the bottom left shows that the ordered distribution of residuals (blue dots)
follows the linear trend of the samples taken from a standard normal distribution with N(0, 1).
This is a decent indication that the residuals are normally distributed.

The residuals over time (top left plot) don't display any obvious seasonality and appear to be white noise.

This is confirmed by the autocorrelation (i.e. correlogram) plot on the bottom right,
which shows that the time series residuals have low correlation with lagged versions of itself.
Those observations lead us to conclude that our model produces a satisfactory fit that could help us
understand our time series data and forecast future values.
"""